In [97]:
import re
import gensim
from gensim.models.phrases import Phrases,Phraser
from gensim.models.word2vec import Word2Vec
from nltk.corpus import stopwords

In [98]:
badwords = stopwords.words('russian')
badwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])

In [99]:
#ради скорости решил посмотреть на преступлении и наказании
input_fname = 'pr_i_nac.txt'
sentences = []
with open(input_fname, 'r') as f:
    for line in f:
        s = re.split('[.!?\-\]\[]', line.strip().lower())
        for sentence in s:
            if len(sentence) > 20:
                sentence = re.sub(r'[,;\(\)«»—]', '', sentence)
                sentence = sentence.strip()
                sentence = ' '.join([x for x in sentence.split() if x not in badwords])
                sentences.append(sentence.strip())

In [100]:
bigram = Phrases(sentences)
bigram_transformer = Phraser(bigram)

In [101]:
def text_generator_bigram():
    for text in sentences:
        yield bigram_transformer[text.split()]

In [102]:
trigram = Phrases(text_generator_bigram())
trigram_transformer = Phraser(trigram)
def text_generator_trigram():
    for text in sentences:
        yield trigram_transformer[bigram_transformer[text.split()]]
        

In [103]:
model = Word2Vec(size=50, window=7, min_count=5, workers = 12)
model.build_vocab(text_generator_trigram())

In [104]:
model.train(text_generator_trigram(), total_examples=model.corpus_count, epochs=1000)

(53807, 90762)

In [122]:
model.wv.most_similar('упал')

[('узнал', 0.49820640683174133),
 ('могу', 0.48999834060668945),
 ('преступления', 0.48643696308135986),
 ('всему', 0.48538464307785034),
 ('тихо', 0.4824371337890625),
 ('квартира', 0.47275683283805847),
 ('человек', 0.4714996814727783),
 ('женщина', 0.4701743721961975),
 ('тревоге', 0.4647528827190399),
 ('плакать', 0.46132224798202515)]